## **Turi Activity Classifier**

Welcome to the activity classification model quickstart on Skafos! The purpose of this notebook is to get you going end-to-end. Below we will do the following:

1. Load activity training data generated from Apple watchOS devices.
2. Build an activity classification model.
3. Validate the model against some test data.
4. Convert the model to CoreML format and sae it to the Saves framework.

The example is based on [Turi Create's Activity Classifier](https://apple.github.io/turicreate/docs/userguide/activity_classifier/) and is associated with this [3-part blog series](https://medium.com/metis-machine/activity-classification-for-watchos-part-1-542d44388c40).

---

Execute each cell one-by-one, by selecting the cell and do one of the following:

-  Clicking the "play" button at the top of this frame.
-  Typing 'Control + Enter' or 'Shift + Enter'.

In [ ]:
# If this is your first time in the JupyterLab workspace - install external dependencies
from utilities.dependencies import install
install(timeout=500)

# No need to do this in the future for this notebook

In [ ]:
# Import necessary libraries
from skafossdk import *
import turicreate as tc

In [ ]:
# Initialize Skafos
ska = Skafos()

### 1. **Load the data**
The data for this example comes from watchOS devices. Motion-sensory readings from the device's accelerometer and gyroscope were collected during a live experiment where three activities took place: walking, sitting, and standing. Read more about the experiment and the data cleaning process [here](https://medium.com/metis-machine/activity-classification-for-watchos-part-1-542d44388c40). You can also find the raw data in the `...skafos.example.data/ActivityClassifier/raw/..` bucket on s3.

Once loaded, the data is split into train and test sets, where 80% of the data is used for training, and 20% is used for model evaluation.

In [ ]:
# Set the column names for the dataframe
activity_data_columns = {
    "motionRotationRateX(rad/s)": "rotation_x",
    "motionRotationRateY(rad/s)": "rotation_y",
    "motionRotationRateZ(rad/s)": "rotation_z",
    "motionUserAccelerationX(G)": "acceleration_x",
    "motionUserAccelerationY(G)": "acceleration_y",
    "motionUserAccelerationZ(G)": "acceleration_z",
    "sessionId": "session_id",
    "activity": "activity"
}

# Loading the dataset from S3 Bucket to SFrame
s3_url = "https://s3.amazonaws.com/skafos.example.data/ActivityClassifier/cleaned/watch_activity_data.csv"
data = tc.SFrame.read_csv(s3_url)[list(activity_data_columns.keys())].rename(activity_data_columns)

In [ ]:
# Make a train-test split such that each data slice contains whole session chunks
train_data, test_data = tc.activity_classifier.util.random_split_by_session(data, session_id='session_id', fraction=0.8)

In [ ]:
# Take a look at the data
train_data.head(5)

### 2. **Build the model**
We use the `tc.activity_classifier.create` function and specify the data, target variable (the activity label), session_id, and a few other arguments needed to properly train the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/activity_classifier/).

In [ ]:
# Create an activity classifier model
model = tc.activity_classifier.create(
    dataset=train_data,
    session_id='session_id',
    target='activity',
    prediction_window=30,
    max_iterations=30,
    batch_size=64
)

---
**Prediction Window** = *number_seconds_between_predictions* x *sample_frequency*

-  *number_seconds_between_predictions* is how many seconds you want between each prediction. Varies by use-case.
-  *sample_frequency* is how many sensor readings are taken each second (by the application).

For this example, the motion-sensor data was collected at 10Hz (or 10 times per second). If we want our model to give an activity prediction every 3 seconds, we should set the prediction window to 30 ~ (3 * 10).

### 3. **Model Validation**
Below we test the model by checking predictions against held out data reserved for evaluation. We find sample data for an activity and see if the model is able to classify it correctly.

In [ ]:
# Get the most common activity in the test sample
activity = test_data['activity'].value_counts()[0]['value']

# Find a session id in the test sample with that activity
session_id = test_data[test_data['activity'] == activity]['session_id'].value_counts()[0]['value']

# Grab a slice of data constrained to a single experiment and activity
test_slice = test_data[(test_data['session_id'] == session_id) & (test_data['activity'] == activity)]

ska.log(f'Evaluating trained activity classifier against {activity} sample data within session {session_id}.', labels=['validation'])

In [ ]:
# Model predictions over the test slice
model.predict(test_slice, output_frequency='per_window')

In [ ]:
# Calculate accuracy of the model against the entire hold out testing set
accuracy = tc.evaluation.accuracy(test_data['activity'], model.predict(test_data))
ska.log(f'The activity classifier predicted {accuracy*100} % of the testing observations correctly!', labels=['validation'])

### 4. **Deliver the model**
Once your model has been created, it must be converted to CoreML and saved to the Skafos framework. Once saved, if you wish to push to your iOS devices, you can use the `.deliver()` method below. We've left that commented out for now.

In [ ]:
# Specify the CoreML model name
model_name = 'ActivityClassifier'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

# Save model asset to Skafos
ska.asset_manager.save(
    name=model_name,              # Name used to load or deliver asset, also used within the Swift SDK.
    files=coreml_model_name,      # File or list of files to bundle together as a versioned asset.
    tags=['latest'],              # User-defined tags to help distinguish your asset.
    access='public'               # Asset access- public/private.
)

In [ ]:
# Deliver asset to devices (push)
#ska.asset_manager.deliver(
#  name=model_name,                # Name used to load or deliver asset, also used within the Swift SDK.
#  tag='latest',                   # User-defined tags to help distinguish your asset.
#  dev=True                        # Push asset through Apple's APNS dev or prod server
#)